In [1]:
from BinaryLayers import *

In [15]:
batch_size = 128
alpha = .15
epsilon = 1e-4
num_units = 100
n_hidden_layers = 1
num_epochs = 30
dropout_in = 0
dropout_hidden = 0

deterministic = False
H = 1.

learning_rate = .001
min_learning_rate = 3e-6
decay = (learning_rate - min_learning_rate) / num_epochs

device = torch.device("cuda")

In [16]:
t = transforms.Compose(
    [
       transforms.ToTensor(),
       transforms.Normalize(mean=(0), std=(1))
    ]
)

dl_train = DataLoader(
    torchvision.datasets.MNIST(
        "/data/mnist",
        download=True,
        train=True,
        transform=t,
        target_transform=torchvision.transforms.Compose([
            lambda x:torch.LongTensor([x]), # or just torch.tensor
            lambda x:torch.nn.functional.one_hot(x, 10)
        ])
    ),
    batch_size=batch_size,
    drop_last=True,
    shuffle=True
)
dl_valid = DataLoader(
    torchvision.datasets.MNIST(
        "/data/mnist",
        download=True,
        train=False,
        transform=t,
        target_transform=torchvision.transforms.Compose([
            lambda x:torch.LongTensor([x]), # or just torch.tensor
            lambda x:torch.nn.functional.one_hot(x, 10)
        ])
    ),
    batch_size=batch_size,
    drop_last=True,
    shuffle=True
)
num_in = 28 * 28

layers = []
layers.append(torch.nn.Dropout(dropout_in))
for i in range(n_hidden_layers):
    layers.append(BinaryDense(num_in, num_units, H=H, deterministic=deterministic))
#     layers.append(torch.nn.BatchNorm1d(num_units))
#     layers.append(torch.nn.Linear(num_in, num_units))
    layers.append(torch.nn.Sigmoid())
#     layers.append(torch.nn.Dropout(dropout_hidden))
    num_in = num_units
# layers.append(torch.nn.Linear(num_in, 10))
layers.append(BinaryDense(num_in, 10, H=H, deterministic=deterministic))
# layers.append(torch.nn.BatchNorm1d(10, eps=epsilon, momentum=alpha))
layers.append(torch.nn.Softmax(1))
model = torch.nn.Sequential(*layers).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=30)
# optimizer = torch.optim.Adam(model.parameters(), lr=3)
lossfunction = torch.nn.MSELoss()

losses = [0] * num_epochs
val_losses = [0] * num_epochs
total_steps = len(dl_train) * num_epochs


for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    ect = 0
    ecv = 0
    ett = 0
    etv = 0
    for i, (input, target) in enumerate(dl_train):
        if i % 10 == 0:
            print(i, end=" ")
        optimizer.zero_grad()
        input = torch.reshape(input, (-1, 28 * 28)).to(device)
        target = torch.reshape(target, (-1, 10)).to(device)
        output = model(input)
        
        loss = lossfunction(output, target.float())
        losses[epoch] += loss.item()
        loss.backward()
        
        optimizer.step()

    model.eval()
    with torch.no_grad():
        for j, (input, target) in enumerate(dl_valid):
            input = torch.reshape(input, (-1, 28*28)).to(device)
            target = target.reshape((-1, 10)).to(device)
            output = model(input)
            loss = lossfunction(output, target.float())
            val_losses[epoch] += loss.item()
    
    print("")
    print("Epoch training loss" , losses[epoch] / len(dl_train))
    print("Epoch valid loss" , val_losses[epoch] / len(dl_valid))


Epoch 1
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 
Epoch training loss 0.1508486959605645
Epoch valid loss 0.12568042484613565
Epoch 2
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 
Epoch training loss 0.09959910818749768
Epoch valid loss 0.07486347954433697
Epoch 3
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 
Epoch training loss 0.059625983166580014
Epoch valid loss 0.048246434006171346
Epoch 4
0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 
Epoch training loss 0.04

In [17]:
tot_acc = 0
with torch.no_grad():
    for j, (input, target) in enumerate(dl_valid):
        input = torch.reshape(input, (-1, 28*28)).to(device)
        target = target.to(device)
        output = model(input)
        target = target.argmax(-1).reshape(-1)
        output = output.argmax(-1).reshape(-1)
        
        tot_acc = (tot_acc * j + int(sum(target == output)) / len(target)) / (j + 1)

print("Validation Accuracy:", tot_acc)

Validation Accuracy: 0.8980368589743589
